In [46]:
from imdb import IMDb
import pandas as pd

In [2]:
ia = IMDb()

In [5]:
matrix_list = ia.search_movie("The Matrix")

In [14]:
ia.update(matrix_list[0])

In [24]:
matrix_list[0]['year']

1999

In [25]:
matrix_list[0]["genres"]

['Action', 'Sci-Fi']

In [35]:
matrix_list[0].getID()

'0133093'

In [36]:
matrix_list[0]["runtimes"]

['136']

In [42]:
def get_imdb(movie_name, ia):
    movie_list = ia.search_movie(movie_name)
    movie = movie_list[0]
    ia.update(movie) 
    print(movie, " - ", movie["year"])
    return movie

In [44]:
pulp_fiction = get_imdb("Pulp Fiction", ia)

Pulp Fiction 1994


In [47]:
df = pd.read_csv("../results/all-matches-with-seeds.csv")

In [59]:
all_movies = df["player_1_name"].append(df["player_2_name"]).unique()

In [64]:
all_movies = pd.DataFrame({"Movie": all_movies})

In [73]:
# Remove Young Frankenstein
all_movies["Movie"] = all_movies["Movie"].apply(lambda x: x.split(" / ")[0])

In [75]:
all_movies["imdb"] = all_movies["Movie"].apply(lambda x: get_imdb(x, ia))

Pulp Fiction 1994
Out of Sight 1998
Goodfellas 1990
Toy Story 1995
Schindler's List 1993
Fight Club 1999
Chungking Express 1994
Barton Fink 1991
Jackie Brown 1997
The Silence of the Lambs 1991
Heat 1995
Boogie Nights 1997
Terminator 2: Judgment Day 1991
Dazed and Confused 1993
Malcolm X 1992
The Big Lebowski 1998
Groundhog Day 1993
Miller's Crossing 1990
Magnolia 1999
Unforgiven 1992
Being John Malkovich 1999
Trainspotting 1996
The Matrix 1999
Hoop Dreams 1994
Fargo 2014
L.A. Confidential 1997
Before Sunrise 1995
Eyes Wide Shut 1999
Rushmore 1998
Reservoir Dogs 1992
The Shawshank Redemption 1994
Se7en 1995
The Godfather 1972
The Apartment 1960
Taxi Driver 1976
Citizen Kane 1941
The Godfather: Part II 1974
Alien 1979
Mulholland Drive 2001
Casablanca 1942
Blue Velvet 1986
Chinatown 1974
Jaws 1975
Raiders of the Lost Ark 1981
2001: A Space Odyssey 1968
Raging Bull 1980
Apocalypse Now 1979
Do the Right Thing 1989
Back to the Future 1985
Star Wars 1977
One Flew Over the Cuckoo's Nest 1975
G

In [82]:
# Three mistakes: Lion King, Safe, Fargo
import pickle

with open("../results/imdb_list.pkl", "wb+") as imdb_list:
    pickle.dump(all_movies["imdb"].tolist(), imdb_list)


In [117]:
imdbs = all_movies["imdb"].tolist()
imdbs[0]["title"]

'Pulp Fiction'

In [119]:
for index, imdb in enumerate(imdbs):
    if imdb["title"] in ("The Lion King", "Safe", "Fargo"):
        print(imdb, index)

Fargo 24
Safe 63
The Lion King 68


In [120]:
print(len(imdbs))

imdbs = [imdb for imdb in imdbs if imdb["title"] not in ("The Lion King", "Safe", "Fargo")]

print(len(imdbs))

118
115


In [122]:
fargo = ia.search_movie("Fargo")[1]
ia.update(fargo)
imdbs.insert(24, fargo)

In [123]:
safe = ia.search_movie("Safe")[2]
ia.update(safe)
imdbs.insert(63, safe)

In [124]:
lion_king = ia.search_movie("The Lion King")[1]
ia.update(lion_king)
imdbs.insert(68, lion_king)

In [125]:
for index, imdb in enumerate(imdbs):
    if imdb["title"] in ("The Lion King", "Safe", "Fargo"):
        print(imdb, index)

Fargo 24
Safe 63
The Lion King 68


In [205]:
with open("../results/imdb_list.pkl", "wb+") as imdb_list:
    pickle.dump(imdbs, imdb_list)

In [126]:
imdb_ids = [imdb.getID() for imdb in imdbs]

In [127]:
imdb_years = [imdb["year"] for imdb in imdbs]

In [128]:
imdb_runtimes = [imdb["runtimes"][0] for imdb in imdbs]

In [129]:
from sklearn.preprocessing import MultiLabelBinarizer
binner = MultiLabelBinarizer()

In [210]:
imdb_country_codes = pd.DataFrame(binner.fit_transform([imdb["country codes"] for imdb in imdbs]),
            columns=[f"cc.{class_}" for class_ in binner.classes_])

In [214]:
imdb_language_codes = pd.DataFrame(binner.fit_transform([imdb["language codes"] for imdb in imdbs]),
            columns=[f"lc.{class_}" for class_ in binner.classes_])

In [221]:
imdb_colors = [1 if imdb["color info"][0].split("::")[0] == "Color" else 0 for imdb in imdbs]

In [196]:
imdb_certificates = []
for imdb in imdbs:
    for certificate in imdb["certificate"]:
        if "United States" in certificate and "TV" not in certificate:
            imdb_certificates.append(certificate.split(":")[1])
            break

In [199]:
imdb_rating = [imdb["rating"] for imdb in imdbs]

In [201]:
imdb_votes = [imdb["votes"] for imdb in imdbs]

In [207]:
all_movies = all_movies.drop(columns="imdb")

In [216]:
all_movies = pd.concat([all_movies, imdb_country_codes, imdb_language_codes], axis=1)

In [217]:
all_movies["id"] = imdb_ids
all_movies["year"] = imdb_years
all_movies["runtime"] = imdb_runtimes
all_movies["color"] = imdb_colors
all_movies["imdb_rating"] = imdb_rating
all_movies["imdb_votes"] = imdb_votes

In [229]:
imdb_genres = pd.DataFrame(binner.fit_transform([imdb["genres"] for imdb in imdbs]),
            columns=[f"genre.{class_}" for class_ in binner.classes_])
all_movies = pd.concat([all_movies, imdb_genres])

/home/deadhead/.local/share/virtualenvs/filmspotting-2019-VAg7pNyZ/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [230]:
all_movies.to_csv("../results/imdb_results.csv", index=False)

In [233]:
imdbs[0].keys()

['title',
 'kind',
 'year',
 'cast',
 'genres',
 'runtimes',
 'countries',
 'country codes',
 'language codes',
 'color info',
 'aspect ratio',
 'sound mix',
 'certificates',
 'original air date',
 'rating',
 'votes',
 'cover url',
 'plot outline',
 'languages',
 'directors',
 'writers',
 'producers',
 'cinematographers',
 'editors',
 'editorial department',
 'casting directors',
 'production designers',
 'art directors',
 'set decorators',
 'costume designers',
 'make up department',
 'production managers',
 'assistant directors',
 'art department',
 'sound department',
 'special effects',
 'stunts',
 'camera department',
 'animation department',
 'casting department',
 'costume departmen',
 'location management',
 'music department',
 'transportation department',
 'miscellaneous',
 'thanks',
 'akas',
 'writer',
 'director',
 'top 250 rank',
 'production companies',
 'distributors',
 'other companies',
 'plot',
 'synopsis',
 'canonical title',
 'long imdb title',
 'long imdb canonical